In [1]:
import os, shutil, random
from pathlib import Path
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
projections_root = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections'
subset_root = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000'
os.makedirs(subset_root, exist_ok=True)

# Create Subset

Create subset of data with 1,000 samples per class (n_arms). This will result in 7,000 samples total per default, 2ds, and phips sub-folders. 21,000 samples total. 

In [31]:
%%time
def reservoir_sample_files(folder_path, k, seed=42):
    """Randomly sample k file paths from a directory with reservoir sampling."""
    random.seed(seed)
    reservoir = []
    with os.scandir(folder_path) as it:
        for i, entry in enumeratecd sy(it):
            if not entry.is_file():
                continue
            if len(reservoir) < k:
                sample = entry.name.rsplit('-',1)[0]
                reservoir.append(sample)
            else:
                j = random.randint(0, i)
                if j < k:
                    sample = entry.name.rsplit('-',1)[0]
                    reservoir[j] = sample
    return reservoir

def create_subset(source_dir, dest_dir, 
    num_samples_per_subfolder=10, 
    seed=42
):
    random.seed(seed)
    source_dir = Path(source_dir)
    dest_dir = Path(dest_dir)
    # Make sure destination directory exists
    dest_dir.mkdir(parents=True, exist_ok=True)
    prefix_dict = {}
    first_folder = True
    # Loop through top-level categories
    for top_folder in source_dir.iterdir():
        if top_folder.is_dir():
            view_type = str(top_folder).rsplit('/', 1)[1]
            # Now loop through second-level folders
            for sub_folder in top_folder.iterdir():
                if sub_folder.is_dir():
                    n_arms = str(sub_folder).rsplit('/', 1)[1]
                    if first_folder==True:
                        selected_files_prefix = reservoir_sample_files(sub_folder, 
                                                                num_samples_per_subfolder, 
                                                                seed)
                        prefix_dict[n_arms] = selected_files_prefix
                    selected_files_prefix = prefix_dict[n_arms]
                    selected_files = [(f+f'-{view_type}.png') for f in selected_files_prefix]
                    # Create destination subfolder
                    rel_path = sub_folder.relative_to(source_dir)
                    target_subfolder = dest_dir / rel_path
                    target_subfolder.mkdir(parents=True, exist_ok=True)
                    # Copy files
                    for filename in selected_files:
                        file_path = os.path.join(sub_folder, filename)
                        shutil.copy(file_path, target_subfolder)
                    print(f"Copied {len(selected_files)} files to {target_subfolder}")
            first_folder=False # only need prefix list the first time
# main
create_subset(
    source_dir=projections_root, 
    dest_dir=subset_root, 
    num_samples_per_subfolder=1000
)

Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/6
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/5
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/8
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/9
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/7
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/10
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/phips/4
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/default/6
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/default/5
Copied 1000 files to /glade/derecho/scratch/joko/synth-ros/params_20

# Now re-organize the directory structure for the subset dataset

In [25]:
def flat_split_dataset(
    source_dir,
    dest_dir,
    split_ratio=(0.7, 0.15, 0.15),
    seed=42,
    valid_exts={".jpg", ".jpeg", ".png"}
):
    random.seed(seed)
    source_dir = Path(source_dir)
    dest_dir = Path(dest_dir)

    assert sum(split_ratio) == 1.0, "Split ratios must sum to 1.0"

    class_dirs = [d for d in source_dir.iterdir() if d.is_dir()]

    # Create split folders
    for split in ["train", "val", "test"]:
        (dest_dir / split).mkdir(parents=True, exist_ok=True)

    for class_dir in class_dirs:
        class_name = class_dir.name
        print(f"Processing class: {class_name}")

        # Get image files
        image_files = [f for f in class_dir.iterdir() if f.suffix.lower() in valid_exts]
        # print(image_files)
    
        # Split into train and temp (val + test)
        train_files, temp_files = train_test_split(
            image_files,
            train_size=split_ratio[0],
            random_state=seed,
            shuffle=True,
        )

        # Split temp into val and test
        val_ratio = split_ratio[1] / (split_ratio[1] + split_ratio[2])
        val_files, test_files = train_test_split(
            temp_files,
            train_size=val_ratio,
            random_state=seed,
            shuffle=True,
        )

        split_map = {
            "train": train_files,
            "val": val_files,
            "test": test_files
        }

        for split, files in split_map.items():
            for file_path in files:
                dest_path = dest_dir / split / file_path.name
                shutil.copy(file_path, dest_path)

        print(f"  → {len(train_files)} train, {len(val_files)} val, {len(test_files)} test")

In [26]:
flat_split_dataset(
    source_dir="/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000/default",
    dest_dir="/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000_default_split"
)

Processing class: 9
  → 700 train, 150 val, 150 test
Processing class: 6
  → 700 train, 150 val, 150 test
Processing class: 5
  → 700 train, 150 val, 150 test
Processing class: 8
  → 700 train, 150 val, 150 test
Processing class: 10
  → 700 train, 150 val, 150 test
Processing class: 7
  → 700 train, 150 val, 150 test
Processing class: 4
  → 700 train, 150 val, 150 test


# Create labels file

In [2]:
# load data with labels
ros_data = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/data/ros-data-merged.txt'
df = pd.read_csv(ros_data)
df.head()

,id,a,c,f_r0,f_hp,f_h0,n_arms,sa,vol,sa_eff,rho_eff
0,35000,13.817545,23.540122,0.867449,0.961225,0.943525,7,33247.371151,181231.947690,0.576030,0.138993
1,35001,13.817545,23.540122,0.867449,0.961225,0.943525,7,33022.961732,184536.198465,0.560433,0.137205
2,35002,13.817545,23.540122,0.867449,0.961225,0.943525,7,33530.598131,187678.178569,0.585745,0.145727
3,35003,13.817545,23.540122,0.867449,0.961225,0.943525,7,32592.037777,166089.393327,0.543729,0.120358
4,35004,13.817545,23.540122,0.867449,0.961225,0.943525,7,32892.678757,179534.751064,0.583751,0.142747


In [3]:
print(df.columns)

Index(['id', 'a', 'c', 'f_r0', 'f_hp', 'f_h0', 'n_arms', 'sa', 'vol', 'sa_eff',
       'rho_eff'],
      dtype='object')


In [4]:
str(df.n_arms[0])

'7'

In [5]:
# create labels
labels = []
# get all file paths in a list 
ds_dir = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000_default_split'
ds_path = Path(ds_dir)
all_files = list(ds_path.rglob("*"))  # includes all files and folders
all_files = [f for f in all_files if f.is_file() and f.suffix.lower() == ".png"] # only png files
all_files[0]

PosixPath('/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000_default_split/test/ros-projection-010913-002-default.png')

In [11]:
test = all_files[0]
test.as_posix().rsplit('/',2)[1]

'test'

In [13]:
# create labels
labels = []
# get all file paths in a list 
ds_dir = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/subset_n1000_default_split'
ds_path = Path(ds_dir)
all_files = list(ds_path.rglob("*"))  # includes all files and folders
all_files = [f for f in all_files if f.is_file() and f.suffix.lower() == ".png"] # only png files
def get_labels(filename, split, id, df): 
    data = df[df.id==int(id)].iloc[0]
    n_arms = str(int(data['n_arms'])) # string 
    rho_eff = data['rho_eff']
    sa_eff = data['sa_eff']
    return [filename, split, n_arms, rho_eff, sa_eff]
# get labels for each files and append
for f in all_files:
    filename = f.name
    id = filename.split('-')[2]
    split = f.as_posix().rsplit('/', 2)[1]
    record = get_labels(filename, split, id, df)
    labels.append(record)
# turn into dataframe and save as csv 
colnames = ['filename', 'split', 'n_arms', 'rho_eff', 'sa_eff']
df_labels = pd.DataFrame(labels, columns=colnames)
savepath = os.path.join(ds_dir, 'labels.csv')
df_labels.to_csv(savepath, index=False)

In [14]:
# test read
df_labels = pd.read_csv(savepath)
df_labels.head()

,filename,split,n_arms,rho_eff,sa_eff
0,ros-projection-010913-002-default.png,test,5,0.015800,0.166602
1,ros-projection-016265-058-default.png,test,5,0.057668,0.324473
2,ros-projection-001142-087-default.png,test,4,0.014152,0.147579
3,ros-projection-017988-092-default.png,test,5,0.100758,0.443553
4,ros-projection-057113-091-default.png,test,9,0.026300,0.280913


# Scratch

In [44]:
from PIL import Image
test_file = '/glade/derecho/scratch/joko/synth-ros/params_200_50_20250403/projections/phips/4/ros-projection-000000-000-phips.png'
image = Image.open(test_file)

In [45]:
print(image.size)
print(image.mode)

(224, 224)
RGB
